In [55]:
import os
import sys

import numpy as np
import magenpy as mgp
import pandas as pd
import matplotlib.pyplot as plt

import xarray
import zarr
import dask.array as da
import scipy
import subprocess

The following code is for selecting a number of SNPS, since the whole dataset has 21 million.

In [56]:
out_markers_file = "/nfs/scistore17/robingrp/adepope/ssgVAMP/selected_snps"

In [57]:
bim_columns = ['Chromosome', 'SNP_ID', 'Genetic_Distance', 'BP_Position', 'Allele_1', 'Allele_2']
bimfile = pd.read_table("/nfs/scistore17/robingrp/adepope/ssgVAMP/all_phase3.bim", names=bim_columns)

/tmp/ipykernel_1722562/3845978015.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  bimfile = pd.read_table("/nfs/scistore17/robingrp/adepope/ssgVAMP/all_phase3.bim", names=bim_columns)


In [58]:
bimfile.shape

(21754024, 6)

In [65]:
num_snps_to_select = 500000
index = np.random.choice(np.arange(bimfile.shape[0]), num_snps_to_select, replace=False)
index.sort()
selected_snps = bimfile["SNP_ID"].to_numpy()[index].flatten()
markers_file = f"{out_markers_file}_{int(num_snps_to_select/1000)}K.txt"
np.savetxt(markers_file, selected_snps, fmt='%s')

We have saved the markers in a txt file. The following code is for calling plink and creating the files with only the SNPs we have selected. For this to work you have to put the plink executable on the bin folder and give it writing and reading permissions. 

In [ ]:
bfile_name = "/nfs/scistore17/robingrp/adepope/ssgVAMP/all_phase3"
output_name = f"/nfs/scistore17/robingrp/adepope/ssgVAMP/1000g_{int(num_snps_to_select/1000)}K"
markers_file = f"{out_markers_file}_{int(num_snps_to_select/1000)}K.txt"
plink_command = ["plink", "--bfile", bfile_name, "--extract", markers_file, "--make-bed","--out", output_name]

process = subprocess.Popen(plink_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

stdout, stderr = process.communicate()

## error handling
if process.returncode == 0:
    pass
else:
    print("PLINK command encountered an error.")
    print("Standard Output:")
    print(stdout)
    print("Standard Error:")
    print(stderr)

Now we load this data with the magenpy package to create the LD matrix. I couldn't get it to work with plink. But if we want to do it later we should load the data with `backend="plink"`.

In [23]:
## indicate where the plink executables are
mgp.set_option("plink1.9_path", "/nfs/scistore07/clustersw/shared/plink/1.90/plink")
mgp.set_option("plink2_path", "/mnt/nfs/clustersw/shared/plink/220415/plink2")

In [66]:
#gdl = mgp.GWADataLoader('/nfs/scistore17/robingrp/avillanu/1000genomes/1000g_20K', backend='plink')
gdl = mgp.GWADataLoader('/nfs/scistore17/robingrp/adepope/ssgVAMP/all_phase3_500k', backend='plink')

> Reading BED file...


The following command creates one matrix for every chromosome. I don't think there is a way to do it for every pair of markers with this package. But we could try to do it with plink directly. In this case using `estimator='sample'` we are computing the value for every pair of markers in the chromosome.

In [67]:
gdl.compute_ld(estimator='windowed',
               output_dir='/nfs/scistore17/robingrp/adepope/ssgVAMP/', window_size=100)

Computing LD matrices:   0%|          | 0/22 [00:00<?, ?it/s]/mnt/nfs/clustersw/Debian/bullseye/JupyterPython/2.0/lib/python3.9/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Computing LD matrices:   5%|▍         | 1/22 [00:19<06:53, 19.68s/it]/mnt/nfs/clustersw/Debian/bullseye/JupyterPython/2.0/lib/python3.9/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Computing LD matrices:   9%|▉         | 2/22 [00:40<06:41, 20.09s/it]/mnt/nfs/clustersw/Debian/bullseye/JupyterPython/2.0/lib/python3.9/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Computing LD matrices:  14%|█▎        | 3/22 [00:57<05:56, 18.76s/it]/mnt/nfs/clustersw/Debian/bullseye/JupyterPython/2.0/lib/python3.9/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Computing LD matrices:  18%|█▊        | 4/22 [01:14<05:23, 17.98s/it]/mnt/nfs/clustersw/Debian/bullseye/JupyterPython/2.0/lib/python3.9/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


...

Computing LD matrices:  77%|███████▋  | 17/22 [03:36<00:40,  8.10s/it]/mnt/nfs/clustersw/Debian/bullseye/JupyterPython/2.0/lib/python3.9/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Computing LD matrices:  82%|████████▏ | 18/22 [03:43<00:30,  7.70s/it]/mnt/nfs/clustersw/Debian/bullseye/JupyterPython/2.0/lib/python3.9/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Computing LD matrices:  86%|████████▋ | 19/22 [03:48<00:21,  7.08s/it]/mnt/nfs/clustersw/Debian/bullseye/JupyterPython/2.0/lib/python3.9/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Computing LD matrices:  91%|█████████ | 20/22 [03:54<00:13,  6.69s/it]/mnt/nfs/clustersw/Debian/bullseye/JupyterPython/2.0/lib/python3.9/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
Computing LD matrices:  95%|█████████▌| 21/22 [03:58<00:05,  5.93s/it]

/mnt/nfs/clustersw/Debian/bullseye/JupyterPython/2.0/lib/python3.9/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
Computing LD matrices: 100%|██████████| 22/22 [04:02<00:00, 11.03s/it]

Now we convert every matrix into a sparse matrix with the `to_csr_matrix()` command. We can put these matrices together in a "diagonal" matrix by blocks. 

In [68]:
matrices = [m.to_csr_matrix() for m in gdl.ld.values()]

In [82]:
LD_matrix = scipy.sparse.tril(scipy.sparse.block_diag(matrices), format="csr")

In [83]:
scipy.sparse.save_npz("/nfs/scistore17/robingrp/adepope/ssgVAMP/LDmatrix_500K_windowed_ws100.npz", LD_matrix)

In [84]:
LD_matrix.shape, LD_matrix.dtype

((428171, 428171), dtype('float64'))

Everything is correct. Then if we convert it to a numpy array and check the size...

In [110]:
#print(sys.getsizeof(LD_matrix.toarray()) * 1e-9, " GB")

We can also save the matrix in a npz file (it was the format I found in the documentation). Then the size is bigger.

In [86]:
print(os.path.getsize("/nfs/scistore17/robingrp/adepope/ssgVAMP/LDmatrix_500K_windowed_ws100.npz")*1e-6, " MB")

296.04994899999997  MB


We plot the matrix to see if it makes any sense.

In [107]:
corrs = np.abs((LD_matrix[12500:13000,12500:13000]).toarray())
plt.figure(figsize=(3,3), dpi=400)
plt.imshow(corrs)
plt.show()
#plt.spy(LD_matrix[1000:2000,1000:2000])


It makes sense to have blocks by chromosome.